# Fine-tuning data prep

Explore some open-source dataset for fine-tuning a small 3B / 7B model.

Topics involved:
- Dataset download and loading
- Preprocessing : prepare input and output based on FT objective
- Tokenization : tokenize input and output
- Batching : prepare batches for training
- Train / Test split

## Data Prep

For most finetuning tasks, the dataset consists of `input` -> `output` pairs, where the `input` is the prompt and the `output` is the expected response. 
An example of this kind of dataset is the [openassistant-guanaco](https://huggingface.co/datasets/timdettmers/openassistant-guanaco?row=0) dataset where `input` is the human prompt and `output` is the assistant response.

For preference tuning using DPO or RLHF tasks the dataset consists of a comparison pairs of `prompt`, `accepted_target` and `rejected_target` pairs. RLHF datasets only contain `accepted_target` and `rejected_target` pairs with the same prompt.

References 
- [Transformers - Data Preparation for LLM finetuning](https://huggingface.co/docs/autotrain/en/llm_finetuning)
- [Fine-tuning LLMs in 2024](https://www.philschmid.de/fine-tune-llms-in-2024-with-trl)


Goal: Download a few sample datasets and print them out.


1. [OpenAssistant](https://huggingface.co/datasets/timdettmers/openassistant-guanaco?row=0)

In [1]:
from datasets import load_dataset

In [3]:
oa_dataset = load_dataset("timdettmers/openassistant-guanaco", split='train')
oa_dataset

Repo card metadata block was not found. Setting CardData to empty.


Dataset({
    features: ['text'],
    num_rows: 9846
})

In [5]:
oa_dataset[0]['text']

'### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, lead